In [ ]:
import glob
import os
from dataclasses import dataclass, field

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pytz
from astral import LocationInfo
from astral.sun import sun
from scipy.stats import norm, cauchy

import strat_defs # custom functions
import prep_data

In [ ]:
# Set up prepd_data data frame
indicator_config = prep_data.IndicatorConfig(ticker='GEV', target='Adj Close')

prepd_data = prep_data.prep_data(config=indicator_config, drop_tickers=False)

In [ ]:
vol_cols = [col for col in prepd_data.columns if 'Volume_' in col]

column_sums = prepd_data[vol_cols].sum().sort_values(ascending=False)

fig = px.bar(x=column_sums.index.str.replace('Volume_', ''), y=column_sums.values,
             labels={'x': 'Ticker', 'y': 'Total Volume'}, title="Total Volume by Ticker")

fig.show()

In [ ]:
column_sums.tail(10)

In [ ]:
data = prepd_data['Daily_Return'].dropna()

# Compute histogram using numpy to get exact bin edges and bin width
hist_values, bin_edges = np.histogram(data, bins=300, density=False)
bin_width = bin_edges[1] - bin_edges[0]

# x values covering the histogram's range using the same bin edges
x = np.linspace(bin_edges[0], bin_edges[-1], 100)

# Create a histogram in Plotly using explicit binning to match np.histogram
hist = go.Histogram(x=data, xbins=dict(start=bin_edges[0], end=bin_edges[-1], size=bin_width), name="Histogram")

# Fit distributions and create PDFs
mu, std = norm.fit(data)
loc, scale = cauchy.fit(data)

pdf_norm = norm.pdf(x, mu, std) * len(data) * bin_width
pdf_cauchy = cauchy.pdf(x, loc, scale) * len(data) * bin_width

# Create curves
curve_norm = go.Scatter(x=x, y=pdf_norm, mode='lines', name="Fitted Normal")
curve_cauchy = go.Scatter(x=x, y=pdf_cauchy, mode='lines', name="Fitted Cauchy")

fig = go.Figure([hist, curve_norm, curve_cauchy])
fig.update_layout(title=f"{indicator_config.ticker} Daily Return Histogram with Fitted Normal and Cauchy Distributions",
                  xaxis_title="Value", yaxis_title="Frequency")
fig.show()